In [ ]:
from pandas import Series, DataFrame
import pandas as pd

In [ ]:
# Special packages
import statsmodels.api as sm
from patsy import dmatrices

Q1: 

In [15]:
data = pd.read_csv('http://users.stat.ufl.edu/~winner/data/clinton1.dat', sep= '\s\s+', header=None, names= ['County Name','Percent voting for Clinton in 1992','Median Age','Mean Savings($)','PerCapita Income  ($)', 'Percent in Poverty', 'Percent Veterans', 'Percent Female','Population Density','Percent in Nursing Homes', 'Crime Index (Per capita)'], index_col = ['County Name'], engine = 'python')
data

,Percent voting for Clinton in 1992,Median Age,Mean Savings($),PerCapita Income ($),Percent in Poverty,Percent Veterans,Percent Female,Population Density,Percent in Nursing Homes,Crime Index (Per capita)
County Name,,,,,,,,,,
"Autauga, AL",30.92,31.7,57623,15768,15.2,10.74,51.41,60.4,2.36,457
"Baldwin, AL",26.24,35.5,84935,16954,13.6,9.73,51.34,66.5,5.40,282
"Barbour, AL",46.36,32.8,83656,15532,25.0,8.82,53.03,28.8,7.02,47
"Blount, AL",32.92,34.5,61249,14820,15.0,9.67,51.15,62.4,2.36,185
"Bullock, AL",67.67,31.7,75725,11120,33.0,7.08,50.76,17.6,2.91,141
"Butler, AL",47.43,33.5,75887,12015,27.1,9.48,53.25,28.1,7.98,266
"Calhoun, AL",38.44,32.7,76073,15162,18.0,11.95,51.66,190.8,4.81,560
"Chambers, AL",45.36,34.8,40498,13887,18.8,10.52,52.77,62.1,6.15,218
"Cherokee, AL",53.61,36.3,56386,13936,15.4,9.29,50.82,35.8,5.61,51


Q2: Predict percentVoting on all regressors. One of the fields in the data is “Percent voting for Clinton in 1992”; let’s call it percentVoting. Regress percentVoting on all available regressors. Which two regressors
have the worst p-values?

In [22]:
df1 = data.rename(columns = {'Percent voting for Clinton in 1992':'percentVoting'})

y, X = dmatrices("percentVoting ~ Q('Median Age') + Q('Mean Savings($)') + Q('PerCapita Income  ($)') + Q('Percent in Poverty') + Q('Percent Veterans') + Q('Percent Female') + Q('Population Density') + Q('Percent in Nursing Homes') + Q('Crime Index (Per capita)')", data=df1, return_type='dataframe')

model = sm.OLS(y, X)       # Set up the model
result = model.fit() 

#get the results
result.pvalues.sort_values(ascending = False)[:2]



Q('Median Age')                  0.238619
Q('Crime Index (Per capita)')    0.141950
dtype: float64

Q3: A formula-creating function. Write a function called formula that takes a list of regressors as input, and outputs the formula for regressing percentVoting on these regressors. For example, formula([’age’, ’savings’]) should output percentVoting ∼ age + savings.

In [30]:
def formula(regressors):
    
    string_to_pass = "percentVoting ~ "
    for i in range(len(regressors)): 
        #for last item in list, don't add a plus sign
        if i == (len(regressors)-1):
            string_to_pass = string_to_pass + "Q('" + regressors[i] + "')"
        else:
            string_to_pass = string_to_pass + "Q('" + regressors[i] + "') + "
            

    

    #get line to return 
    return string_to_pass

test = ['age','savings','income']

teststring = formula(test)
print(teststring)


percentVoting ~ Q('age') + Q('savings') + Q('income')


Q4: R-squared computation function. Write a function called rsquared that, given a list of regressors, returns the R-squared corresponding to a regression of percentVoting with respect to the list of regressors. You can use the formula function you wrote above.

In [38]:
def rsquared(regressors):
    
    string = formula(regressors)
    
    #create design matrix
    y, X = dmatrices(string, data=df1, return_type='dataframe')
    
    model = sm.OLS(y, X)       # Set up the model
    results = model.fit() 
    
    return results.rsquared


test = ['Percent in Poverty','Population Density']

print("R-Squared Value")
print(rsquared(test))
    
    

0.261463311348
R-Squared Value
0.285829802291


Q5: Find the next best regressor. Write a function called bestnextregressor that, given a list of regressors (called current regressor list) and the list of all possible regressors (called all regressors list), picks one regressor from all regressors list that, when combined with the regressors in current regressor list, yields the highest R-square. The function should return this regressor, and the corresponding R-square.

In [ ]:
def no_overlaps(current,all_):
    
    for item in current:
        if item in all_:
            all_.remove(item)
    return all_
        

def bestnextregressor(current_regressor_list, all_regressors_list):
    
    #find unique regressors 
    
    best_regressor = ''
    max_r_square = 0.000000000000000000000000000003
    
    regressors_to_compare = no_overlaps(current_regressor_list, all_regressors_list)
    
    for item in regressors_to_compare:
        
        current_regressor_list.append(item)
        
        #call r_square
        temp_r_square = rsquared(current_regressor_list)
        
        #check if max
        if temp_r_square > max_r_square:
            best_regressor = item
            max_r_square = temp_r_square
            
        current_regressor_list.remove(item)
        
            
    return (best_regressor,max_r_square)
            
    
#test function

current_regressor_list = ['Percent in Poverty','Median Age']
all_regressor_list = ['Median Age', 'Mean Savings($)', 'PerCapita Income  ($)', 'Percent in Poverty', 'Percent Veterans','Percent Female','Population Density','Percent in Nursing Homes','Crime Index (Per capita)']

print(bestnextregressor(current_regressor_list, all_regressor_list)[0])
print(bestnextregressor(current_regressor_list, all_regressor_list)[1])
        
        
        
        
        
        
    
    

Q6: Pick the k best regressors. Write a function called best regressors that takes two arguments:
• num regressors, which is the number of regressors we want, and
• all regressors list, which again is the list of all possible regressors.
The function should compute the best num regressors  by picking them one at a time using best next regressor that you wrote above. The function should return a tuple of the following items:
• a list of the best regressors, in the order in which they were picked, and
• the corresponding R-square values.
In other words, you want a list of the results of calling best next regressor.

In [ ]:
# def bestregressors(num_regressors,all_regressors_list):
    
#     current_reg_list = []
#     r_square_values = []
#     num_rounds = 0
#     print('here')
#     while (num_rounds < 3):
        
#         tuple_reg = bestnextregressor(current_reg_list,all_regressors_list)
#         current_reg_list.append(tuple_reg[0])
#         r_square_values.append(tuple_reg[1])
        
#         print(current_reg_list)
#         num_rounds += 1
        
#     return (current_reg_list,r_square_values)

# #testing my function
# all_regressor_list = ['Median Age', 'Mean Savings($)', 'PerCapita Income  ($)', 'Percent in Poverty', 'Percent Veterans','Percent Female','Population Density','Percent in Nursing Homes','Crime Index (Per capita)']
# tuple_reg2 = bestregressors(3,all_regressor_list)
# print(tuple_reg2[0])
# print(tuple_reg2[1])
print('here')
    
    